# Battle of Neighborhoods: How to open a coffee shop in Toronto
### By analyzing the neighborhoods in Toronto, we decide to find the best spot for opening a coffee shop
## Introduction
As the world's economy growing so rapidly, thousands of coffe shop are opening everywhere in the city. How to choose a good location to stay competitive and attract customers is critical when opening a new business. By using clustering to analyze neighborhoods and studying the ratings of other restaurants within the neighborhoods, I will draw conclusion on the most suitable location. In this project, I will gather the geographical data, analytical information from Foursquare API. And neighborhoods information is collected through wikipedia. K-means clustering method is used for analyzing purposes and results are displayed through folium maps for visualization.


### 1. Data gathering the processing

#### import libraries

In [2]:
#install libraries 
!pip install geopy    
!pip install folium   
!pip install geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt # for graphical usage 

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap

import pandas as pd
import requests

print('Libraries imported.')

     |████████████████████████████████| 102kB 9.6MB/s ta 0:00:011
     |████████████████████████████████| 102kB 10.6MB/s ta 0:00:01
Libraries imported.


#### getting coordinates for each neighborhood

In [3]:
# neighborhoods data download from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
ls = pd.read_html(url)
df=ls[0]
df=df[df.Borough != 'Not assigned']
# read geographical coordinates from cvs
coordinates=pd.read_csv('http://cocl.us/Geospatial_data')
data = pd.merge(df,coordinates[['Postal Code','Latitude','Longitude']],on='Postal Code',how='left')
data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Toronto maps with neighborhoods 

In [6]:
#Geographical coordinates
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 2. Foursquare API data collecting and Neighborhood analysis
#### Collect neighborhood venues from Foursquare API

In [9]:
# APL credentials
CLIENT_ID = 'FINEY221XOE51ZF3WKRR1LG1BHMLR5XEKI5RN4UJ4H3UX4UZ' # your Foursquare ID
CLIENT_SECRET = 'LN0MANDFEELRUPSP3D4BE1VSNNA2GFF05YXOMZVZ535Q4TFP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FINEY221XOE51ZF3WKRR1LG1BHMLR5XEKI5RN4UJ4H3UX4UZ
CLIENT_SECRET:LN0MANDFEELRUPSP3D4BE1VSNNA2GFF05YXOMZVZ535Q4TFP


In [13]:
LIMIT = 100
def getNeighbourhoodVenues( latitude, longitude,neighbourhood, radius=1000 ):
    venues = []
    for lat, long, neighborhood in zip(latitude, longitude ,neighbourhood):

        # create the API request URL
        APIurl = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            long,
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(APIurl).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        for venue in results:
            venues.append((
                neighborhood,
                lat, 
                long, 
                venue['venue']['name'], 
                venue['venue']['id'], 
                venue['venue']['location']['lat'], 
                venue['venue']['location']['lng'],  
                venue['venue']['location']['distance'],  
                venue['venue']['categories'][0]['name']))
    
    # convert the venues list into a DataFrame
    venues = pd.DataFrame(venues)
    # define the column names
    venues.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueId', 'VenueLatitude', 'VenueLongitude','VenueDistance','VenueCategory']
            
    return venues 

neighbourhood_venues = getNeighbourhoodVenues (data['Latitude'], data['Longitude'], data['Neighbourhood'] )

print('There are {} unique venue categories. Some of them are as below:'.format(len(neighbourhood_venues['VenueCategory'].unique())))
neighbourhood_venues.head()

There are 331 unique venue categories. Some of them are as below:


,Neighbourhood,Latitude,Longitude,VenueName,VenueId,VenueLatitude,VenueLongitude,VenueDistance,VenueCategory
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,4b8991cbf964a520814232e3,43.759840,-79.324719,833,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,4e8d9dcdd5fbbbb6b3003c7b,43.751976,-79.332140,245,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,57e286f2498e43d84d92d34a,43.760668,-79.326368,866,Café
3,Parkwoods,43.753259,-79.329656,A&W,58a8dcaa6119f47b9a94dc05,43.760643,-79.326865,852,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,4bafa285f964a5203a123ce3,43.746143,-79.324630,889,Grocery Store


In [14]:
neighbourhood_venues.groupby('Neighbourhood').count()

,Latitude,Longitude,VenueName,VenueId,VenueLatitude,VenueLongitude,VenueDistance,VenueCategory
Neighbourhood,,,,,,,,
Agincourt,47,47,47,47,47,47,47,47
"Alderwood, Long Branch",27,27,27,27,27,27,27,27
"Bathurst Manor, Wilson Heights, Downsview North",33,33,33,33,33,33,33,33
Bayview Village,16,16,16,16,16,16,16,16
"Bedford Park, Lawrence Manor East",42,42,42,42,42,42,42,42
Berczy Park,100,100,100,100,100,100,100,100
"Birch Cliff, Cliffside West",14,14,14,14,14,14,14,14
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",49,49,49,49,49,49,49,49


In [15]:
print('Top 20 distinct venue counts are as below')
neighborhood_venues[['VenueId','VenueCategory']].drop_duplicates().groupby('VenueCategory').count()[['VenueId']].rename(columns={"VenueId": "Count"}).sort_values(by=['Count'], ascending=False)[:20]

Top 20 distinct venue counts are as below


,Count
VenueCategory,
Coffee Shop,270
Café,137
Park,129
Pizza Place,121
Restaurant,99
Bakery,85
Grocery Store,81
Italian Restaurant,77
Sandwich Place,73


###  2. Analyzing each neighbourhood

#### we need to create a new dataframe displaying top 10 venues in each neighbourhood

In [18]:
# one hot encoding
data_onehot = pd.get_dummies(neighbourhood_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
data_onehot['Neighbourhood'] = neighbourhood_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [data_onehot.columns[-1]] + list(data_onehot.columns[:-1])
data_onehot = data_onehot[fixed_columns]

data_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soc

In [19]:
data_grouped = data_onehot.groupby('Neighbourhood').mean().reset_index()
data_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soc

In [20]:
num_top_venues = 5

for hood in data_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = data_grouped[data_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.13
1         Shopping Mall  0.09
2                Bakery  0.04
3  Caribbean Restaurant  0.04
4        Sandwich Place  0.04


----Alderwood, Long Branch----
               venue  freq
0     Discount Store  0.11
1        Pizza Place  0.07
2  Convenience Store  0.07
3           Pharmacy  0.07
4       Dance Studio  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0  Convenience Store  0.06
1        Pizza Place  0.06
2        Coffee Shop  0.06
3               Bank  0.06
4               Park  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.12
1  Japanese Restaurant  0.12
2          Gas Station  0.12
3        Grocery Store  0.12
4                Trail  0.06


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.07
1         Pizza Place  0.07
2         Coffee Shop  0.07
3          Restaurant  0.05
4  

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = data_grouped['Neighbourhood']

for ind in np.arange(data_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(data_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Sandwich Place,Bakery,Sri Lankan Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Supermarket
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Convenience Store,Pharmacy,Intersection,Sandwich Place,Liquor Store,Garden Center,Gas Station,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Convenience Store,Pizza Place,Park,Coffee Shop,Bank,Ski Chalet,Fried Chicken Joint,Sushi Restaurant,Supermarket,Bridal Shop
3,Bayview Village,Bank,Gas Station,Grocery Store,Japanese Restaurant,Restaurant,Trail,Chinese Restaurant,Skating Rink,Café,Intersection
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Pizza Place,Coffee Shop,Bank,Restaurant,Sandwich Place,Fast Food Restaurant,Bridal Shop,Cosmetics Shop,Bakery


### 3. K-means clustering

In [54]:
# set number of clusters
kclusters = 5

data_grouped_clustering = data_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([3, 3, 3, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [71]:
labels=int(kmeans.labels_)
labels

array([3, 3, 3, 0, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 0, 1, 1, 1, 0,
       1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 0, 3, 3, 1, 1, 3, 3, 3, 1, 1, 3, 3,
       1, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 2, 0, 1, 1, 3, 3, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 0, 1, 1, 3, 3, 1, 1, 4], dtype=int32)

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster labels', labels)

data_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
data_merged = data_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

data_merged.head() # check the last columns!

In [79]:
data_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Shop & Service,Supermarket,Food & Drink Shop,Pizza Place,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Portuguese Restaurant,Men's Store,French Restaurant,Lounge,Golf Course,Grocery Store,Gym / Fitness Center,Park,Hockey Arena
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Café,Theater,Diner,Restaurant,Bakery,Breakfast Spot,Gym / Fitness Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Fried Chicken Joint,Dessert Shop,Furniture / Home Store,Vietnamese Restaurant,Athletics & Sports,Sushi Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Pizza Place,Park,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Café,Gastropub,Bubble Tea Shop,Burger Joint


In [80]:
data_merged.dropna(inplace=True)

In [81]:
data_merged['Cluster Labels']=data_merged['Cluster Labels'].astype(int)

In [82]:
data_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Shop & Service,Supermarket,Food & Drink Shop,Pizza Place,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Coffee Shop,Portuguese Restaurant,Men's Store,French Restaurant,Lounge,Golf Course,Grocery Store,Gym / Fitness Center,Park,Hockey Arena
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Café,Theater,Diner,Restaurant,Bakery,Breakfast Spot,Gym / Fitness Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Fried Chicken Joint,Dessert Shop,Furniture / Home Store,Vietnamese Restaurant,Athletics & Sports,Sushi Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Pizza Place,Park,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Café,Gastropub,Bubble Tea Shop,Burger Joint


In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighbourhood'], data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4. Examine Clusters

#### cluster one

In [85]:
data_merged.loc[data_merged['Cluster Labels'] == 0, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,North York,0,Pizza Place,Fast Food Restaurant,Grocery Store,Coffee Shop,Gas Station,Pub,Pet Store,Rental Car Location,Restaurant,Park
32,Scarborough,0,Ice Cream Shop,Convenience Store,Fast Food Restaurant,Sandwich Place,Grocery Store,Japanese Restaurant,Train Station,Coffee Shop,Restaurant,Women's Store
34,North York,0,Furniture / Home Store,Coffee Shop,Pizza Place,Bank,Restaurant,Bar,Caribbean Restaurant,Chinese Restaurant,Falafel Restaurant,Japanese Restaurant
39,North York,0,Bank,Gas Station,Grocery Store,Japanese Restaurant,Restaurant,Trail,Chinese Restaurant,Skating Rink,Café,Intersection
51,Scarborough,0,Beach,Ice Cream Shop,Pizza Place,Sports Bar,Park,Restaurant,Hardware Store,Pharmacy,Event Space,Donut Shop
56,York,0,Furniture / Home Store,Grocery Store,Sandwich Place,Discount Store,Gas Station,Wine Shop,Playground,Fast Food Restaurant,Video Store,Dessert Shop
71,Scarborough,0,Pizza Place,Middle Eastern Restaurant,Grocery Store,Burger Joint,Intersection,Soccer Field,Supermarket,Gas Station,Furniture / Home Store,Korean Restaurant
89,Etobicoke,0,Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Park,Beer Store,Hardware Store,Caribbean Restaurant,Bus Line


#### cluster two

In [86]:
data_merged.loc[data_merged['Cluster Labels'] == 1, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Coffee Shop,Portuguese Restaurant,Men's Store,French Restaurant,Lounge,Golf Course,Grocery Store,Gym / Fitness Center,Park,Hockey Arena
2,Downtown Toronto,1,Coffee Shop,Park,Pub,Café,Theater,Diner,Restaurant,Bakery,Breakfast Spot,Gym / Fitness Center
3,North York,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Fried Chicken Joint,Dessert Shop,Furniture / Home Store,Vietnamese Restaurant,Athletics & Sports,Sushi Restaurant
4,Downtown Toronto,1,Coffee Shop,Pizza Place,Park,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Café,Gastropub,Bubble Tea Shop,Burger Joint
7,North York,1,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Supermarket,Bank,Asian Restaurant,Café,Pizza Place
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Gastropub,Italian Restaurant,Plaza,Japanese Restaurant,Pizza Place,Middle Eastern Restaurant,New American Restaurant,Café
11,Etobicoke,1,Park,Hotel,Pizza Place,Restaurant,Mexican Restaurant,Café,Bank,Grocery Store,Gym,Clothing Store
12,Scarborough,1,Breakfast Spot,Playground,Park,Burger Joint,Italian Restaurant,Fish & Chips Shop,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
13,North York,1,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Supermarket,Bank,Asian Restaurant,Café,Pizza Place
14,East York,1,Coffee Shop,Pizza Place,Park,Sandwich Place,Athletics & Sports,Convenience Store,Café,Liquor Store,Skating Rink,Restaurant


#### cluster three

In [87]:
data_merged.loc[data_merged['Cluster Labels'] == 2, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,2,Hotel,Coffee Shop,Drugstore,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


#### cluster four

In [88]:
data_merged.loc[data_merged['Cluster Labels'] == 3, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Shop & Service,Supermarket,Food & Drink Shop,Pizza Place,Fast Food Restaurant
5,Etobicoke,3,Pharmacy,Café,Skating Rink,Convenience Store,Playground,Bank,Bakery,Golf Course,Shopping Mall,Grocery Store
6,Scarborough,3,Fast Food Restaurant,Trail,Coffee Shop,Bakery,Arts & Crafts Store,Chinese Restaurant,Bank,Caribbean Restaurant,Greek Restaurant,Supermarket
8,East York,3,Brewery,Pizza Place,Bank,Athletics & Sports,Breakfast Spot,Fast Food Restaurant,Gastropub,Bakery,Rock Climbing Spot,Coffee Shop
18,Scarborough,3,Pizza Place,Coffee Shop,Fast Food Restaurant,Bank,Beer Store,Chinese Restaurant,Fried Chicken Joint,Filipino Restaurant,Sandwich Place,Greek Restaurant
21,York,3,Park,Bus Stop,Pharmacy,Cosmetics Shop,Café,Bank,Bakery,Discount Store,Falafel Restaurant,Grocery Store
22,Scarborough,3,Coffee Shop,Park,Mobile Phone Shop,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Pharmacy,Drugstore,Dumpling Restaurant,Eastern European Restaurant
26,Scarborough,3,Bakery,Coffee Shop,Bank,Indian Restaurant,Gas Station,Intersection,Caribbean Restaurant,Bus Line,Thai Restaurant,Fried Chicken Joint
27,North York,3,Pharmacy,Coffee Shop,Park,Pizza Place,Intersection,Convenience Store,Shopping Mall,Chinese Restaurant,Sandwich Place,Korean Restaurant
28,North York,3,Convenience Store,Pizza Place,Park,Coffee Shop,Bank,Ski Chalet,Fried Chicken Joint,Sushi Restaurant,Supermarket,Bridal Shop


#### cluster five

In [90]:
data_merged.loc[data_merged['Cluster Labels'] == 4, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Park,Pool,Zoo,Falafel Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


### 5. Discussion

#### Cluster 1(red): 
In this cluster, we can see grocery shops and pizza shops are the most common in the area. However, neighbourhoods in this cluster presents a variety of different venues from the top 10 list. 

#### Cluster 2(purple): 
Coffee shops are the most common venues almost in all neighbourhoods. From the top 10 dataset, we can tell there are mostly entertainment business such as different restaurants, parks, shopping complex and etc. Illustrated from the folium map, this cluster are packed in the centre of Toronto including downtwon Toronto and North York, and not many neighbourhoods astray too far away from the centre. 

#### Cluster 3(blue):
Unfornately there is solely one neighbourhood is classified into this cluster, and it is allocated in Etobicoke where is considered far away from centre of Toronto. The most common venue is surprisingly hotel, which none of other neighbourhoods present this feature. It could be the reason that it is very close to the Pearson airport. Nevertheless, coffee shops are still the second most common venue in the neighbourhood. 

#### Cluster 4(green): 
Presented on the folium map, the majority of the neighbourhoods classified into this cluster are located in North York and Scarborough. Again, coffee shops and restaurants are the top 2 most common venues. 

#### Cluster 5(orange): 
This is a sole neighbourhood cluster again. Even though it is surrounded by a few other neighbourhoods from other clustering, it shows a very strange list of most common venue in the surrending. Top three most common venues are park, pool and zoo. And the list goes on by a few exotic taste restaurants such as Falafel and Ethioplian. Furthermore, coffee shop is not even on the list of top 10 regardless of the fact that is the most popular among all other clusters. 

### 6. Conclusion

From this short analysis, it is obvious that coffee shop is the most common venue in the city of Toronto; therefore it is defitinely not a easy start for a new business. Cluster 2 and 4 are the two clusters that contains the most coffee shop, also these two clusters represents the majority of the neighbourhoods. However, in the rest of the clusters, they might be more appropriate choices when deciding to not compete with the existing coffee shops. 